# Sensitivity Analysis

In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_save_dir = "/cluster/home/kamara/syntax-shap/data"

model_name = "mistral"
threshold_ = 0.5


In [3]:

folder_path = f"/cluster/work/zhang/kamara/syntax-shap/results_ids/scores/{model_name}"  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        if (file.endswith('.pkl')):
            # Extract dataset, model name, and algorithm from the filename
            dataset = parts[-5]
            model_name = parts[-4]
            algorithm = parts[-3]
            seed = parts[-2]
            threshold = parts[-1].split('.pkl')[0]  # Removes the file extension

            if eval(threshold) == threshold_:
                # Construct the full file path
                file_path = os.path.join(folder_path, file)

                # Read the pickle file
                with open(full_path, 'rb') as file:
                    data = pickle.load(file)

                # Assuming the content is a dictionary, create a DataFrame
                df = pd.DataFrame(data)
                # Add the extracted details as new columns
                df['algorithm'] = algorithm
                df['n_token'] = df['input'].str.split().apply(len)
                df['seed'] = seed
                df['dataset'] = dataset

                # Append to the list of dataframes
                df_list.append(df)

# Concatenate all dataframes in the list
df_final_single_threshold = pd.concat(df_list, ignore_index=True)
df_final_single_threshold


,fid_keep_rd,fid_keep,fid_rmv,log_odds_keep,acc_at_k,prob_diff_at_k,input_id,input,algorithm,n_token,seed,dataset
0,0.420129,0.378165,0.375612,-1.961204,0.2,0.781496,94,Dallas is not located in,partition,5,0,negation
1,0.010138,0.010138,0.010138,-9.224166,0.2,0.785604,98,Detroit is not located in,partition,5,0,negation
2,0.184373,0.177081,0.184373,-3.230112,0.4,0.709440,97,Denver is located in,partition,4,0,negation
3,-0.054903,0.020616,0.014170,-0.852201,0.1,0.575964,355,Martha is a aunt with a,partition,6,0,negation
4,0.030357,-0.155401,-0.130865,0.687662,0.7,0.296770,305,A lion sees with,partition,4,0,negation
...,...,...,...,...,...,...,...,...,...,...,...,...
32807,0.004985,0.004985,0.004985,-8.514475,0.1,0.529815,81,Different voices have different musical,syntax,5,1,generics
32808,-0.158138,-0.317924,0.412794,0.571078,0.5,0.021312,2384,School education is given equally in both Ital...,syntax,9,1,generics
32809,0.978595,0.961704,0.985015,-3.583949,0.3,0.807675,1431,Different groups think the word means different,syntax,7,1,generics
32810,0.090099,0.090099,-0.011166,-11.408674,0.1,0.715237,3653,Many people think it is one of the most influe...,syntax,12,1,generics


In [4]:
df = df_final_single_threshold.groupby(['dataset', 'algorithm','seed']).mean().reset_index()

In [10]:
#df = df[df['dataset'] == 'negation']
df.groupby(['dataset', 'algorithm'])['prob_diff_at_k'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.689339       NaN
           partition  0.687029  0.004578
           random     0.688533       NaN
           shap       0.596951       NaN
           syntax     0.614685  0.006755
           syntax-w   0.638033  0.001772
negation   lime       0.577953       NaN
           partition  0.585225  0.010090
           random     0.594707       NaN
           shap       0.581616       NaN
           syntax     0.554645  0.019438
           syntax-w   0.539817  0.020058
rocstories lime       0.659697       NaN
           partition  0.660815  0.007354
           random     0.658436       NaN
           shap       0.591031       NaN
           syntax     0.589297  0.004998
           syntax-w   0.608500  0.004703

In [11]:

pd.pivot_table(df, index=['algorithm'], columns = 'dataset', values='prob_diff_at_k', aggfunc=['mean']).reset_index()

algorithm      mean                     
dataset             generics  negation rocstories
0             lime  0.689339  0.577953   0.659697
1        partition  0.687029  0.585225   0.660815
2           random  0.688533  0.594707   0.658436
3             shap  0.596951  0.581616   0.591031
4           syntax  0.614685  0.554645   0.589297
5         syntax-w  0.638033  0.539817   0.608500

In [12]:
pd.pivot_table(df, index=['algorithm'], columns = 'dataset', values='prob_diff_at_k', aggfunc=['std']).reset_index()

algorithm       std                     
dataset             generics  negation rocstories
0        partition  0.004578  0.010090   0.007354
1           syntax  0.006755  0.019438   0.004998
2         syntax-w  0.001772  0.020058   0.004703

In [6]:
df.groupby(['dataset', 'algorithm'])['fid_keep'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.292510       NaN
           partition  0.288581  0.007735
           random     0.298309       NaN
           shap       0.235039       NaN
           syntax     0.266771  0.002620
           syntax-w   0.274856  0.002984
negation   lime       0.148582       NaN
           partition  0.161750  0.012115
           random     0.156825       NaN
           shap       0.145048       NaN
           syntax     0.161794  0.009881
           syntax-w   0.156275  0.008490
rocstories lime       0.240307       NaN
           partition  0.239162  0.003448
           random     0.235557       NaN
           shap       0.200983       NaN
           syntax     0.214747  0.004547
           syntax-w   0.219870  0.003211